In [4]:
import pandas as pd

def read_stock_tweets_labled():
    df = pd.read_csv('tweets_labelled.csv', sep=';')
    df.dropna(inplace=True)
    df['sentiment'] = df['sentiment'].apply(lambda x:1 if x == 'positive' else 0)
    df = df.rename(columns={"sentiment": "Score"})
    return df

df = read_stock_tweets_labled()
df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1300 entries, 0 to 1299
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1300 non-null   int64 
 1   created_at  1300 non-null   object
 2   text        1300 non-null   object
 3   Score       1300 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 50.8+ KB


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np

def process(df):
    #trying tfidf 
    stemmed_data = [" ".join(SnowballStemmer("english", ignore_stopwords=True).stem(word)  
         for sent in sent_tokenize(message)
        for word in word_tokenize(sent))
        for message in  df['text']]
    print('stemmed')
    vectorizer = TfidfVectorizer(stop_words = 'english', min_df = 4, max_df = 0.8)
    dtm = vectorizer.fit_transform(stemmed_data)
    num_features = 60
    svd = TruncatedSVD(num_features)
    print('here')
    normalizer = Normalizer(copy=False)
    print('this done')
    lsa = make_pipeline(svd, normalizer)
    dtm = lsa.fit_transform(dtm)
    print('shape')
    print(np.shape(dtm))
    dtm = np.transpose(dtm)
    columns = ['svd' + str(i) for i in range(num_features)]
    # svd = pd.DataFrame(dtm, columns=columns)
    # svd.fillna(0)
    # labels= ['SV'+str(i) for i num_features]
    for i in range(num_features):
        name = 'svd' + str(i)
        # print(np.shape(dtm[:][i]))
        df[name] = dtm[i]
        df[name] = df[name].fillna('')

    return df

process(df)
df.info()


stemmed
here
this done
shape
(1300, 60)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1300 entries, 0 to 1299
Data columns (total 64 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          1300 non-null   int64  
 1   created_at  1300 non-null   object 
 2   text        1300 non-null   object 
 3   Score       1300 non-null   int64  
 4   svd0        1300 non-null   float64
 5   svd1        1300 non-null   float64
 6   svd2        1300 non-null   float64
 7   svd3        1300 non-null   float64
 8   svd4        1300 non-null   float64
 9   svd5        1300 non-null   float64
 10  svd6        1300 non-null   float64
 11  svd7        1300 non-null   float64
 12  svd8        1300 non-null   float64
 13  svd9        1300 non-null   float64
 14  svd10       1300 non-null   float64
 15  svd11       1300 non-null   float64
 16  svd12       1300 non-null   float64
 17  svd13       1300 non-null   float64
 18  svd14       1300 non-null   

In [6]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn import utils
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor

# Load files into DataFrames


def get_model(df):

    X_train = df

    # X_train = utils.shuffle(X_train, random_state=1)

    # Split training set into training and testing set
    X_train, X_test, Y_train, Y_test = train_test_split(
            X_train.drop(['Score'], axis=1),
            X_train['Score'],
            test_size=1/4.0,
            random_state=0
        )

    X_train_processed = X_train.drop(columns=['id', 'created_at', 'text'])
    X_test_processed = X_test.drop(columns=['id', 'created_at', 'text'])
    # X_submission_processed = X_submission.drop(columns=['Id', 'Time', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'ProductId', 'UserId', 'Text', 'Summary', 'Score'])

    # print(Y_train.head)
    # Y_train = Y_train.fillna(4) #shouldn't need to do this
    # print(Y_train.isnull().values.any())

    # print(Y_test.head)
    # print(Y_test.isnull().values.any())
    # Y_test = Y_test.fillna(4) #shouldn't need to do this

    print('printing the infos')
    print(X_train_processed.info())
    print(X_test_processed.info())
    # print(X_train_processed.isnull().values.any())


    print((X_train_processed.shape), Y_train.shape)
    print((X_test_processed.shape), Y_test.shape)


    # print(X_train[X_train.columns[0]].head(10))
    # subX_train = (X_train['good_cnt']) / (X_train['good_cnt'] + X_train['bad_cnt'] + X_train['okay_cnt'])
    # # subX_test = X_test[X_test['Score']]
    # plt.scatter(subX_train, Y_train, c="slategray", alpha=0.4, linewidths=0.1)
    # # plt.scatter(subX_test, y_CA_H_test, c="seagreen", alpha=0.2, linewidths=0.3)
    # plt.xlabel('ProductScore')
    # plt.ylabel('Actual Score')

    # plt.show()

    # Predict the score using the model

    # print(X_train['good_cntSummary'])




    # model = sm.OLS(Y_train, X_train_processed)
    # results = model.fit()
    # print(results.summary())
    # Y_test_predictions = model.predict(np.transpose(X_test_processed))

    model = XGBRegressor()
    model.fit(X_train_processed, Y_train)
    Y_test_predictions = model.predict(X_test_processed)


    # print(Y_test_predictions.isnull().values.any())
    # print(Y_test.isnull().values.any())
    print('got the predictions')
    print(len(Y_test_predictions))
    # Y_test_predictions = Y_test_predictions.round()
    # Y_test_predictions = [min(5, max(1, x)) for x in Y_test_predictions] 

    print(str(len(Y_test)) + ' ' + str(len(Y_test_predictions)))

    print("RMSE on testing set = ", mean_squared_error(Y_test, Y_test_predictions))

    # prediction['Score'] = ret

    # submission = prediction[['Id', 'Score']]
    # print(Y_test_predictions.head())
    # Y_test_predictions.to_csv("./data/y_test_submission.csv", index=False)

    # Plot a confusion matrix
    # cm = confusion_matrix(Y_test, Y_test_predictions, normalize='true')
    # sns.heatmap(cm, annot=True)
    # plt.title('Confusion matrix of the classifier')
    # plt.xlabel('Predicted')
    # plt.ylabel('True')
    # plt.show()

    return model

model = get_model(df)



printing the infos
<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 1268 to 684
Data columns (total 60 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   svd0    975 non-null    float64
 1   svd1    975 non-null    float64
 2   svd2    975 non-null    float64
 3   svd3    975 non-null    float64
 4   svd4    975 non-null    float64
 5   svd5    975 non-null    float64
 6   svd6    975 non-null    float64
 7   svd7    975 non-null    float64
 8   svd8    975 non-null    float64
 9   svd9    975 non-null    float64
 10  svd10   975 non-null    float64
 11  svd11   975 non-null    float64
 12  svd12   975 non-null    float64
 13  svd13   975 non-null    float64
 14  svd14   975 non-null    float64
 15  svd15   975 non-null    float64
 16  svd16   975 non-null    float64
 17  svd17   975 non-null    float64
 18  svd18   975 non-null    float64
 19  svd19   975 non-null    float64
 20  svd20   975 non-null    float64
 21  svd21   975 non-n

In [77]:
tweets = pd.read_csv("Tweet.csv")

tweets = tweets.rename(columns={'body': 'text'})
tweets.drop(columns=['writer', 'comment_num','retweet_num','like_num'])

tweets['text'] = tweets['text'].apply(lambda x : '' if not (type(x) == type('a')) else x)

tweets = process(tweets)

tweets.to_csv("tweets_processed.csv", index=False)

tweets.info()   



stemmed
here
this done
shape
(3717964, 60)


In [20]:
import pandas as pd
from datetime import datetime
tweets = pd.read_csv('tweets_processed.csv')

tweets['date'] = tweets['post_date'].apply(lambda x : str(datetime.fromtimestamp(x)))

print(tweets.info())


date = tweets.date.str.split(expand=True)
tweets["DATE"] = date[0]
tweets.sort_values(by=['DATE'])
tweets = tweets[tweets.DATE != 'date']
tweets = tweets[tweets.DATE != '=======']


tweets_dropped = tweets.drop(columns=['date', 'tweet_id', 'writer', 'post_date', 'comment_num', 'retweet_num', 'like_num', 'text', 'DATE'], inplace=False)
#     X_test_processed = X_test.drop(columns=['id', 'created_at', 'text'])
predictions = model.predict(tweets_dropped)

tweets['sent'] = predictions

tweets.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3530306 entries, 0 to 3530305
Data columns (total 68 columns):
 #   Column       Dtype  
---  ------       -----  
 0   tweet_id     int64  
 1   writer       object 
 2   post_date    int64  
 3   text         object 
 4   comment_num  int64  
 5   retweet_num  int64  
 6   like_num     int64  
 7   svd0         float64
 8   svd1         float64
 9   svd2         float64
 10  svd3         float64
 11  svd4         float64
 12  svd5         float64
 13  svd6         float64
 14  svd7         float64
 15  svd8         float64
 16  svd9         float64
 17  svd10        float64
 18  svd11        float64
 19  svd12        float64
 20  svd13        float64
 21  svd14        float64
 22  svd15        float64
 23  svd16        float64
 24  svd17        float64
 25  svd18        float64
 26  svd19        float64
 27  svd20        float64
 28  svd21        float64
 29  svd22        float64
 30  svd23        float64
 31  svd24        float64
 32

In [25]:
stock_data = pd.read_csv("AAPL.csv")

# limit tweets to tweets with over 10 likes

tweets = tweets[tweets.apply(lambda x : int(x['like_num']) > 500 and int(x['comment_num']) > 5, axis=1)]


grouped_df = tweets.groupby(by = ["DATE"])
sentiment = []
date=[]
for name,group in grouped_df:
    date.append(name)
    sentiment.append(group.sent.mean())
stock_df = pd.DataFrame({"Date": date, "Mean_Sentiment": sentiment})

# stock_df['Mean_Sentiment'] = stock_df['Mean_Sentiment'].apply(lambda x  : print(x) if x < 0 else x)
stock_df



,Date,Mean_Sentiment
0,2015-01-27,0.146988
1,2015-03-09,0.467021
2,2015-04-28,0.236669
3,2015-06-24,0.790913
4,2016-01-26,0.562061
...,...,...
211,2019-09-15,0.237849
212,2019-09-16,0.450124
213,2019-09-17,0.509969
214,2019-09-18,0.478774


In [22]:
stock_data["increased"] = np.where(stock_data["Close"]>stock_data["Open"], 1, 0)
stock_data["change_value"] = stock_data["Close"]-stock_data["Open"]
# stock_data['Date'] = stock_data['date']

# print(stock_data)
# print(stock_df)

x = pd.merge(stock_data, stock_df, on="Date")
# print(x)
# x['Mean_Sentiment'] = x.Mean_Sentiment.shift(-1, fill_value=0)
# x = x[x.apply(lambda x : int(x['Date'][:4]) > 2017, axis=1)]
x = x.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
print(x)
print(x.corr())

from scipy.stats import pearsonr

# print(x["change_value"].corr(x["Mean_Sentiment"]))
# print(x["increased"].corr(x["Mean_Sentiment"]))
# print(pearsonr(x["change_value"],x["Mean_Sentiment"]))


           Date  increased  change_value  Mean_Sentiment
0    2014-12-31          0     -0.610001        0.412905
1    2015-01-05          0     -0.510000        1.224695
2    2015-01-08          1      0.664999        0.594274
3    2015-01-09          0     -0.164999        0.606336
4    2015-01-14          1      0.190001        0.587961
..          ...        ...           ...             ...
985  2019-09-16          1      0.542499        0.470423
986  2019-09-17          1      0.184997        0.465139
987  2019-09-18          1      0.427502        0.439712
988  2019-09-19          0     -0.262497        0.345741
989  2019-09-20          0     -0.912502        0.418222

[990 rows x 4 columns]
                increased  change_value  Mean_Sentiment
increased        1.000000      0.718894        0.028701
change_value     0.718894      1.000000        0.053212
Mean_Sentiment   0.028701      0.053212        1.000000
